In [3]:
#The necessary libraries are imported. These include keras for building the deep learning model,
#sklearn for training and evaluating the SVM, and numpy for numerical operations.

from keras.datasets import fashion_mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [4]:
#The Fashion MNIST dataset is loaded using the fashion_mnist.load_data() function from Keras. This returns the training and test data as two tuples.

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [5]:
#The data is preprocessed by reshaping the input images to have a single color channel
#and normalizing the pixel values to be between 0 and 1. The labels are also converted to one-hot encoded vectors using the to_categorical function from Keras.

X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [6]:
#The training data is split into a training set and a validation set. 
#This is done to monitor the model’s performance on unseen data during training and to prevent overfitting.

X_val = X_train[50000:]
y_val = y_train[50000:]
X_train = X_train[:50000]
y_train = y_train[:50000]

In [7]:
#A CNN model is created using the Sequential class from Keras.

model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [8]:
#The model is compiled with the Adam optimizer, categorical cross-entropy loss, and accuracy as a metric.
#It is then fit on the training data for 10 epochs, using the validation data to monitor performance.

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10)

Epoch 1/10
1563/1563 [==============================] - 74s 46ms/step - loss: 0.4737 - accuracy: 0.8282 - val_loss: 0.3457 - val_accuracy: 0.8763
Epoch 2/10
1563/1563 [==============================] - 65s 42ms/step - loss: 0.3117 - accuracy: 0.8860 - val_loss: 0.2927 - val_accuracy: 0.8941
Epoch 3/10
1563/1563 [==============================] - 66s 42ms/step - loss: 0.2654 - accuracy: 0.9036 - val_loss: 0.2781 - val_accuracy: 0.8996
Epoch 4/10
1563/1563 [==============================] - 63s 40ms/step - loss: 0.2351 - accuracy: 0.9130 - val_loss: 0.2597 - val_accuracy: 0.9060
Epoch 5/10
1563/1563 [==============================] - 61s 39ms/step - loss: 0.2082 - accuracy: 0.9236 - val_loss: 0.2616 - val_accuracy: 0.9030
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 0.1826 - accuracy: 0.9327 - val_loss: 0.2807 - val_accuracy: 0.8963
Epoch 7/10
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1640 - accuracy: 0.9382 - val_loss: 0.2478 -

In [9]:
#A new model is created to extract features from the validation set using one of the fully connected layers of the trained CNN model. 
#This is done by creating a new Sequential model and adding all layers of the trained CNN model except for the final output layer.
#The features are extracted from the validation set by calling the predict method on the feature extractor model.


feature_extractor = Sequential()
for layer in model.layers[:-1]:
    feature_extractor.add(layer)
features_val = feature_extractor.predict(X_val)

313/313 [==============================] - 6s 18ms/step


In [10]:
#A linear SVM is trained on these extracted features using the SVC class from sklearn.

svm = SVC(kernel='linear')
svm.fit(features_val, y_val.argmax(axis=1))

SVC(kernel='linear')

In [11]:
#Features are also extracted from the test set using the same feature extractor model.

features_test = feature_extractor.predict(X_test)

313/313 [==============================] - 8s 24ms/step


In [12]:
#The trained SVM is evaluated on these test set features using the accuracy_score function from sklearn.

y_pred = svm.predict(features_test)
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred)
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.9009
